In [6]:
import os

os.environ["OPENAI_API_KEY"] = ''
os.environ["OPENAI_BASE_URL"] = 'https://aihubmix.com/v1'

use_openai = True
QUESTION_GENERATOR_MODEL = "aihubmix-Llama-3-3-70B-Instruct"
ANSWER_GENERATOR_MODEL = "aihubmix-Llama-3-3-70B-Instruct"
THINKING_MODEL = "aihubmix-Llama-3-3-70B-Instruct"

In [7]:
import os
# Remove OPENAI_API_KEY from environment variables
if "OPENAI_API_KEY" in os.environ:
    del os.environ["OPENAI_API_KEY"]

# Set Ollama base URL for local usage
os.environ["OPENAI_BASE_URL"] = "http://192.168.1.134:11434"

use_openai = False

EMBEDDING_MODEL = "bge-m3:latest"
QUESTION_GENERATOR_MODEL = "llama3.3:70b"
ANSWER_GENERATOR_MODEL = "llama3.3:70b"
THINKING_MODEL = "llama3.3:70b"


# 测试连接

In [8]:
from langchain.llms import Ollama

def test_ollama_connection():
    try:
        llm = Ollama(
            base_url=os.environ["OPENAI_BASE_URL"],
            model=QUESTION_GENERATOR_MODEL,  # 或其他你在Ollama中可用的模型
            temperature=0.7
        )
        response = llm.invoke("你好，请告诉我 1 + 1 等于几？")
        print("响应内容：", response)
        return True
    except Exception as e:
        print("连接失败！错误信息：\n", str(e))
        return False

if test_ollama_connection():
    print("✅ 成功连接到 Ollama 服务！")
else:
    print("❌ 连接异常，请检查配置！")


响应内容： 1 + 1 = 2！
✅ 成功连接到 Ollama 服务！


In [9]:
import fitz  # PyMuPDF
import os
import hashlib
import datetime
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List

# --- PDF 解析和元数据提取 ---
def parse_pdf(pdf_folder: str) -> List[dict]:

    attachments = []
    for filename in os.listdir(pdf_folder):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(pdf_folder, filename)
            file_path = os.path.normpath(file_path)  # 规范化路径

            attachment_info = {
                'filename': filename,
                'file_path': file_path,
                'content_type': 'application/pdf',
                'md5': 'N/A',
                'title': 'N/A',
                'author': 'N/A',
                'creation_date': 'N/A',
                'modification_date': 'N/A',
                'error': None,  # 用于记录错误信息
            }

            # 计算 MD5 哈希值
            try:
                with open(file_path, 'rb') as f:
                    data = f.read()
                    attachment_info['md5'] = hashlib.md5(data).hexdigest()
            except Exception as e:
                attachment_info['error'] = f"MD5 计算错误: {e}"

            # 提取 PDF 元数据
            try:
                with fitz.open(file_path) as doc:
                    metadata = doc.metadata
                    attachment_info['title'] = metadata.get('title', 'N/A')
                    attachment_info['author'] = metadata.get('author', 'N/A')

                    # 解析 PDF 日期
                    def parse_pdf_date(date_str):
                        if not date_str:
                            return None  # 处理空字符串
                        if date_str.startswith("D:"):
                            date_str = date_str[2:]
                        try:
                            return datetime.datetime.strptime(date_str, "%Y%m%d%H%M%S%z")
                        except ValueError:
                            try:
                                return datetime.datetime.strptime(date_str, "%Y%m%d%H%M%S")
                            except ValueError:
                                return None
                        except:
                            return None

                    creation_date = parse_pdf_date(metadata.get('creationDate', ''))
                    modification_date = parse_pdf_date(metadata.get('modDate', ''))

                    if creation_date:
                        attachment_info['creation_date'] = creation_date.isoformat()
                    if modification_date:
                        attachment_info['modification_date'] = modification_date.isoformat()

            except Exception as e:
                attachment_info['error'] = f"PDF 处理错误: {e}"

            attachments.append(attachment_info)
    return attachments
attachments = parse_pdf("F:\Postgraduate\work_project\MinerU\ssrf")

In [10]:
try:
    import fitz # PyMuPDF
    import os

    print("\nParsing PDF contents...")

    # 检查PDF文件是否存在并可访问
    valid_attachments = []
    for attachment in attachments:
        if attachment['file_path'] and os.path.exists(attachment['file_path']):
            valid_attachments.append(attachment)
        else:
            print(f"File not found or not accessible: {attachment['filename']}")

    print(f"\nProcessing {len(valid_attachments)} accessible PDF files...")

    # 解析PDF内容
    for idx, attachment in enumerate(valid_attachments):
        try:
            print(f"\n[{idx+1}/{len(valid_attachments)}] Processing: {attachment['filename']}")

            # 打开PDF文件
            doc = fitz.open(attachment['file_path'])

            # 获取基本信息
            print(f"Pages: {len(doc)}")
            print(f"Metadata: {doc.metadata}")

            # 获取文本内容（仅前两页示例）
            max_pages_to_show = min(2, len(doc))
            for page_num in range(max_pages_to_show):
                page = doc[page_num]
                text = page.get_text()
                print(f"\nPage {page_num+1} preview (first 200 chars):")
                print(text[:200] + "..." if len(text) > 200 else text)

            # 关闭文档
            doc.close()

        except Exception as e:
            print(f"Error processing {attachment['filename']}: {str(e)}")

except ModuleNotFoundError:
    print("PyMuPDF (fitz) module not found. Please install it with: pip install pymupdf")
except Exception as e:
    print(f"Error initializing PDF parsing: {str(e)}")


Parsing PDF contents...

Processing 172 accessible PDF files...

[1/172] Processing: 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf
Pages: 7
Metadata: {'format': 'PDF 1.6', 'title': '', 'author': 'CNKI', 'subject': '', 'keywords': '', 'creator': 'ReaderEx_DIS 2.3.0 Build 4002', 'producer': 'TTKN', 'creationDate': "D:20230629183533-08'00'", 'modDate': '', 'trapped': '', 'encryption': None}

Page 1 preview (first 200 chars):
0434002-1
研究论文
第 43 卷 第 4 期/2023 年 2 月/光学学报
14.4 keV 迈克耳孙X 射线干涉仪设计及核心器件
表征
司尚禹
1， 李中亮
1*， 薛莲
1， 杨俊亮
2， 罗红心
1， 李伟
3， 张小威
2**
1中国科学院上海高等研究院上海光源中心，上海 201204；
2中国科学院高能物理研究所多学科中心，北京 100049；
3中国计量科学研究院前沿计量科...

Page 2 preview (first 200 chars):
0434002-2
研究论文
第 43 卷 第 4 期/2023 年 2 月/光学学报
续迈克耳孙干涉仪的集成测试提供了技术积累和器
件基础。
2　迈克耳孙X 射线干涉仪设计
迈克耳孙X 射线干涉仪设计如图1 所示，通过在
一体型LLL 干涉仪中插入一个可通过14. 4 keV 光子
的MDCM，可将无色散的LLL 干涉仪转变成为一台
具有色散能力的迈克耳孙干涉仪。LLL 干涉仪的衍
射晶面指数为...

[2/172] Processing: DEPU对上海光源储存环的影响与补偿.pdf
Pages: 5
Metadata: {'format': 'PDF 1.3', 'title': 'NONE', 'author': 'TOM', 'subject': 'OCR-PDF

In [11]:
valid_attachments
# attachments

[{'filename': '14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf',
  'file_path': 'F:\\Postgraduate\\work_project\\MinerU\\ssrf\\14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf',
  'content_type': 'application/pdf',
  'md5': 'e1a2e229b37fd2326e06447deef166e6',
  'title': '',
  'author': 'CNKI',
  'creation_date': 'N/A',
  'modification_date': 'N/A',
  'error': None},
 {'filename': 'DEPU对上海光源储存环的影响与补偿.pdf',
  'file_path': 'F:\\Postgraduate\\work_project\\MinerU\\ssrf\\DEPU对上海光源储存环的影响与补偿.pdf',
  'content_type': 'application/pdf',
  'md5': 'ad35b39e3da13ab9679477b02243e0f9',
  'title': 'NONE',
  'author': 'TOM',
  'creation_date': 'N/A',
  'modification_date': 'N/A',
  'error': None},
 {'filename': 'Libera Photon处理器在上海光源XBPM中的应用研究.pdf',
  'file_path': 'F:\\Postgraduate\\work_project\\MinerU\\ssrf\\Libera Photon处理器在上海光源XBPM中的应用研究.pdf',
  'content_type': 'application/pdf',
  'md5': '86b008ac665ef02e550152f07ce1bfe0',
  'title': 'NONE',
  'author': 'TOM',
  'creation_date': 'N/A',
  'modification_date': 'N/A',
  

In [12]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI


# 创建LLM
# 使用OpenAI API
if use_openai:
    llm = ChatOpenAI(
        temperature=0.7,
        model_name=QUESTION_GENERATOR_MODEL,
    )
    embeddings = OpenAIEmbeddings()
# 使用Ollama
else:
    from langchain.llms import Ollama
    llm = Ollama(
        base_url=os.environ["OPENAI_BASE_URL"],
        model=QUESTION_GENERATOR_MODEL,  # 或其他你在Ollama中可用的模型
        temperature=0.7
    )
    from langchain.embeddings import OllamaEmbeddings
    embeddings = OllamaEmbeddings(base_url=os.environ["OPENAI_BASE_URL"],model=EMBEDDING_MODEL)

def process_pdf_and_generate_questions(pdf_path):
    print(f"\n处理PDF: {os.path.basename(pdf_path)}")

    # 加载PDF
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()
    print(f"已加载 {len(documents)} 页内容")

    # 分割文本
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_documents(documents)
    print(f"文档已分割为 {len(chunks)} 个块")


    vectorstore = FAISS.from_documents(chunks, embeddings)

    # 创建检索器
    retriever = vectorstore.as_retriever(search_kwargs={"k": len(chunks)})

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )

    # 生成问题
    prompt = """
# 问答生成

## 背景

上海光源（Shanghai Synchrotron Radiation Facility，简称SSRF）是中国大陆第一台中能第三代同步辐射光源，坐落在浦东张江高科技园区，由国家、中国科学院和上海市人民政府共同投资建设，于2004年12月25日开工建设，2009年4月29日竣工，5月6日正式对用户开放。上海光源首批建成7条光束线站；2015年～2018年期间，"梦之线"、蛋白质设施5线6站、SiP·ME2研究平台等陆续建成。2023年9月，上海光源线站工程顺利通过工艺验收，标志着项目全部建设任务圆满完成，光束线站、用户辅助实验室、加速器等各系统全面转入试运行阶段。目前，上海光源共有34条光束线46个实验站向用户开放运行，总体性能位居国际先进水平。上海光源具有波长范围宽、高强度、高亮度、高准直性、高偏振与准相干性、可准确计算、高稳定性等一系列优异的特性，可用以从事生命科学、材料科学、环境科学、信息科学、凝聚态物理、原子分子物理、团簇物理、化学、医学、药学、地质学等多学科的前沿基础研究，以及微电子、医药、石油、化工、生物工程、医疗诊断和微加工等高技术的开发应用的实验研究。十四年来，上海光源围绕科学前沿、国家重大需求与产业核心问题支撑用户开展创新研究，以工程建设、运行开放、关键技术研发和科学研究四位一体可持续发展的总思路，聚焦重大基础科学突破和关键核心技术发展，为广大用户提供了一个跨学科、综合性、多功能的大科学研究平台，在生命科学、凝聚态物理、材料科学、化学、能源与环境科学等多个学科前沿基础研究和高新技术研发领域产生了一批具有国际影响力的研究成果，有力地推动了相关学科的发展，成为我国科学家参与国际竞争的强大助力。同时，也进一步推动了我国包括同步辐射光源、自由电子激光在内的大科学装置的发展。上海光源是国家重大创新能力基础设施，是支撑众多学科前沿基础研究、应用研究、高新技术研发的大型综合实验研究平台，向各领域用户开放申请。上海光源每年向用户供光4000～5000小时，所有用户均可通过"申请-评审-批准"程序获得上海光源实验机时。目前，中国科学院上海高等研究院负责上海光源的运行、维护和性能升级。

## 任务

你是一个高中生，正在参加上海光源的夏令营。请基于给定的论文内容，生成5-8个你想问上海光源资深工程师的问题。

## 要求

1. 直接输出问题列表，每个问题一行，以数字编号
2. 不要包含任何前言、解释或结尾
3. 问题应该是通用的，不要直接引用论文（不要使用"文中"、"论文中"等字样）
4. 问题应聚焦于上海光源的技术、应用和历史，不要询问未来发展
5. 问题必须使用中文，严禁输出任何英文问题
6. 问题应该具有教育意义，适合高中生的知识水平
7. 问题应该涵盖上海光源的基本原理、应用领域和技术特点
8. 问题应该能引发思考，不要过于简单的是非题

## 示例输出格式
1. 什么是储存环全相干光源？
2. 工作点反馈系统对同步辐射光源有什么作用？
3. 超导波振荡器降温特性是如何进行研究的？
"""
    result = qa_chain({"question": prompt, "chat_history": []})

    print("\n生成的问题:")
    print(result["answer"])

    return result


C:\Users\lamonki\AppData\Local\Temp\ipykernel_38232\1885502528.py:26: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(base_url=os.environ["OPENAI_BASE_URL"],model=EMBEDDING_MODEL)


## 构建问题

In [13]:
DEBUG = False

In [14]:
# 处理所有PDF并生成问题集
import json
try:
    print("\n处理所有PDF文件并生成问题集...")

    # 添加变量标志是否重新生成问题
    REGENERATE_QUESTIONS = False
    questions_file_path = "./pdf_questions.jsonl"

    all_questions = {}

    # 检查是否存在已保存的问题集且不需要重新生成
    if os.path.exists(questions_file_path) and not REGENERATE_QUESTIONS:
        print(f"从本地文件加载已有问题集: {questions_file_path}")
        # 读取JSONL文件，每行是一个JSON对象
        with open(questions_file_path, 'r', encoding='utf-8') as f:
            all_questions = {}
            for line in f:
                entry = json.loads(line.strip())
                pdf_name = entry.get("pdf_name", "")
                question = entry.get("question", "")
                if pdf_name:
                    if pdf_name not in all_questions:
                        all_questions[pdf_name] = []
                    if question:
                        all_questions[pdf_name].append(question)
        print(f"已加载 {len(all_questions)} 个PDF的问题集")
        # 打印加载的前10个问题
        print("\n前10个问题示例:")
        count = 0
        for pdf_name, questions in all_questions.items():
            for question in questions:
                print(f"- {pdf_name}: {question}")
                count += 1
                if count >= 10:
                    break
            if count >= 10:
                break
    else:
        # 根据DEBUG标志决定处理的PDF数量
        pdf_to_process = valid_attachments[:2] if DEBUG else valid_attachments

        for attachment in pdf_to_process:
            pdf_path = attachment['file_path']
            pdf_name = os.path.basename(pdf_path)

            try:
                print(f"\n正在处理: {pdf_name}")
                result = process_pdf_and_generate_questions(pdf_path)

                # 从结果中提取问题
                if isinstance(result, str):
                    questions_text = result
                else:
                    # 如果agent返回的是字典类型的结果
                    questions_text = result.get('answer', "未能提取问题")

                # 拆分问题，每个问题独占一条记录
                questions_list = []
                # 按行分割，然后处理每一行
                for line in questions_text.strip().split('\n'):
                    line = line.strip()
                    # 跳过空行
                    if not line:
                        continue
                    # 跳过可能的标题行、介绍性文字或非问题内容
                    # 使用正则表达式判断是否是以数字+点开头的问题
                    import re
                    # 过滤掉不是以数字+点开头的行，或者是一些介绍性文字
                    if not re.match(r'^\d+\.\s', line) or \
                        "Here are" in line or "以下是" in line or "engineers" in line or "Facility" in line or "The questions are" in line:
                        continue

                    # 如果是有效问题，只保留问题主体，去掉前面的数字编号
                    line = re.sub(r'^\d+\.\s', '', line)
                    # 添加到问题列表
                    questions_list.append(line)

                # 将每个问题单独存储
                all_questions[pdf_name] = questions_list

                print(f"已完成 {pdf_name} 的问题生成，共 {len(questions_list)} 个问题")

            except Exception as e:
                print(f"处理 {pdf_name} 时出错: {str(e)}")
                all_questions[pdf_name] = [f"处理出错: {str(e)}"]

    # 生成JSONL格式，每个问题独占一行
    print(f"\n生成JSONL格式问题集: {questions_file_path}")
    with open(questions_file_path, 'w', encoding='utf-8') as f:
        for pdf_name, questions_list in all_questions.items():
            # 确保questions_list是列表
            if isinstance(questions_list, str):
                questions_list = [questions_list]

            # 为每个问题创建一个单独的JSON对象并写入JSONL文件
            for question in questions_list:
                if question.strip():
                    qa_entry = {
                        "pdf_name": pdf_name,
                        "question": question.strip(),
                        "answer": "",
                        "sources": []
                    }
                    f.write(json.dumps(qa_entry, ensure_ascii=False) + '\n')

    print(f"已将问题集保存为JSONL格式: {questions_file_path}")

except Exception as e:
    print(f"处理PDF文件集时出错: {str(e)}")



处理所有PDF文件并生成问题集...
从本地文件加载已有问题集: ./pdf_questions.jsonl
已加载 188 个PDF的问题集

前10个问题示例:
- 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf: 上海光源作为一台中能第三代同步辐射光源，其主要技术优势在哪里？
- 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf: 同步辐射光源与自由电子激光相比，有什么不同之处？
- 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf: 上海光源的光束线站如何为各个学科的研究提供支持？
- 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf: 什么是上海光源的波长范围，以及它如何应用于材料科学等领域？
- 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf: 上海光源在生命科学、凝聚态物理和化学等方面有哪些典型的应用案例？
- 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf: 上海光源的加速器系统是如何设计和维护的，以保证其高稳定性和准确性？
- 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf: 用户如何申请并使用上海光源的实验机时，并且需要满足什么条件？
- 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf: 上海光源在推动我国大科学装置发展方面扮演了怎样的角色？
- 3维准直测量技术在上海光源中的应用研究.pdf: 上海光源的同步辐射光束线站主要用于哪些领域的研究？
- 3维准直测量技术在上海光源中的应用研究.pdf: 什么是上海光源中加速器的作用，以及它如何产生高能电子束？

生成JSONL格式问题集: ./pdf_questions.jsonl
已将问题集保存为JSONL格式: ./pdf_questions.jsonl


## 构建智能体，PDF向量化

In [16]:
try:
    print("\n构建基于所有PDF的RAG智能体...")

    # 添加向量rebuild标志
    REBUILD_VECTOR = False
    vector_store_path = "./all_pdfs_vectorstore"

    # 创建一个新的向量存储，包含所有有效PDF的内容
    all_pdfs_vectorstore = None

    # 导入re模块
    import re

    def is_reference(text):
        # 检查是否包含引用标题
        if re.search(r'\b(参考文献|引用文献|参考书目|references|bibliography|citations)\b', text.lower()):
            return True

        # 检查是否包含引用格式
        # 英文格式：(Author, 2023)
        if re.search(r'\([A-Za-z]+\s*,\s*\d{4}\)', text):
            return True
        # 中文格式：(张三, 2023)
        if re.search(r'\([\u4e00-\u9fa5]+\s*,\s*\d{4}\)', text):
            return True

        # 检查是否包含引用关键词
        reference_keywords = [
            "references", "bibliography", "citations", "cited by",
            "source:", "retrieved from", "et al.", "pp.", "vol.",
            "journal", "conference", "proceedings",
            "参考文献", "引用文献", "参考书目", "引自", "摘自", "参见",
            "等人", "第", "卷"
        ]
        for keyword in reference_keywords:
            if keyword in text.lower():
                return True

        return False




    # 检查是否需要重建向量存储
    if REBUILD_VECTOR or not os.path.exists(vector_store_path):
        if valid_attachments:
            print("正在构建新的向量存储...")
            # 处理所有PDF文件并合并到一个向量存储中
            all_documents = []
            for attachment in valid_attachments:
                pdf_path = attachment['file_path']
                try:
                    print(f"处理PDF: {pdf_path}")
                    loader = PyMuPDFLoader(pdf_path)
                    documents = loader.load()
                    # 添加元数据以标识来源
                    for doc in documents:
                        doc.metadata["source"] = pdf_path
                    all_documents.extend(documents)
                except Exception as e:
                    print(f"处理PDF时出错 {pdf_path}: {str(e)}")

            # 创建文本分割器
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )

            # 分割文档
            all_splits = text_splitter.split_documents(all_documents)

            filtered_splits = [doc for doc in all_splits if not is_reference(doc.page_content)]

            # 创建向量存储
            all_pdfs_vectorstore = FAISS.from_documents(filtered_splits, embeddings)

            # 保存向量存储到本地
            all_pdfs_vectorstore.save_local(vector_store_path)
            print(f"向量存储已保存到: {vector_store_path}")
        else:
            print("没有有效的PDF文件可供处理")
    else:
        print(f"正在加载本地向量存储: {vector_store_path}")
        # 初始化嵌入模型        # 从本地加载向量存储，添加allow_dangerous_deserialization=True参数
        all_pdfs_vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
        print("本地向量存储加载完成")

    if all_pdfs_vectorstore:
        # 创建检索器
        all_pdfs_retriever = all_pdfs_vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 10}
        )

        # 创建问答链
        all_pdfs_qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=all_pdfs_retriever,
            return_source_documents=True
        )
        print("创建问答链完成")

except Exception as e:
    print(f"构建RAG智能体时出错: {str(e)}")
    import traceback
    traceback.print_exc()



构建基于所有PDF的RAG智能体...
正在加载本地向量存储: ./all_pdfs_vectorstore
本地向量存储加载完成
创建问答链完成


## 生成问题答案

In [17]:
from langchain.prompts import PromptTemplate

# 定义分类 Prompt
classification_prompt_template = """你将收到一个问题，请根据以下八类对其进行分类。仅返回类别名称，不要解释。

类别定义：
1. Open QA（开放式问答）：问题没有固定答案，回答需要基于广泛的知识或推理。示例：“为什么天空是蓝色的？”
2. General QA（通用问答）：问题涉及常识或通用知识，回答通常是事实性的。示例：“中国的首都是哪里？”
3. Classification（分类）：问题要求对输入进行分类或判断。示例：“这篇文章的情感是积极的还是消极的？”
4. Closed QA（封闭式问答）：问题有明确的、固定的答案。示例：“本文提到的作者是谁？”
5. Brainstorming（头脑风暴）：问题需要创造性思维或多角度的回答。示例：“如何提高公司的创新能力？”
6. Information Extraction（信息抽取）：问题要求从文本中提取特定信息。示例：“本文提到的日期是什么时候？”
7. Summarization（摘要）：问题要求对文本进行总结或概括。示例：“请用一句话总结这篇文章的主要内容。”
8. Creative Writing（创意写作）：问题需要生成创造性的文本，如故事、诗歌等。示例：“写一个关于未来城市的故事。”

question: {question}
"""

classification_prompt = PromptTemplate(
    template=classification_prompt_template,
    input_variables=["question"]
)

from langchain.chains import LLMChain

# 构建分类问答链
classification_chain = LLMChain(
    llm=llm,
    prompt=classification_prompt
)


# 定义 Prompt 模板
prompt_template = """你是一个入职上海同步辐射光源多年的专家，最近在带刚刚入职的大学生，你需要为他浅显易懂的讲解一些问题,仅用一段话讲明重点就行。
## 要求
1. 答案中不要出现“根据资料”、“根据文献”、“根据提供的文本”等词语
2. 不要出现表格等形式，输出格式为纯文本,如存在公式，形式为LaTeX格式

请基于下文回答问题:

{context}

question: {question}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

C:\Users\lamonki\AppData\Local\Temp\ipykernel_38232\4211579870.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  classification_chain = LLMChain(


In [18]:
# 使用RAG生成回答并保存为jsonl格式
try:
    import json

    from langchain.prompts import PromptTemplate

    # 定义输出文件
    qa_output_file = "pdf_qa_results.jsonl"

    if all_pdfs_qa_chain and all_questions:
        print("\n回答生成的问题并保存为JSONL格式:")

        # 打开文件准备写入
        with open(qa_output_file, 'w', encoding='utf-8') as f:
            # 遍历每个PDF文件的问题
            for pdf_name, questions_text in all_questions.items():
                print(f"\n处理来自 {pdf_name} 的问题:")

                # 尝试将问题文本分割成单独的问题
                # questions = questions_text

                for question in questions_text:
                    # 清理问题文本
                    clean_question = re.sub(r'^\d+\.\s*', '', question).strip()
                    if clean_question and len(clean_question) > 10:  # 确保问题有足够长度
                        print(f"\nquestion: {clean_question}")

                        # 对问题进行分类
                        category = classification_chain.run(question=clean_question)
                        print(f"分类: {category}")

                        # answer_result = all_pdfs_qa_chain.invoke({"question": clean_question, "chat_history": []})
                        answer_result = all_pdfs_qa_chain.invoke({
                            "question": clean_question,
                            "chat_history": [],  # 如果需要，保留聊天历史
                            "prompt": prompt  # 传入 PromptTemplate 对象
                        })
                        print(f"回答: {answer_result['answer']}")

                        # 提取检索到的文档内容（context）
                        context_list = []
                        for doc in answer_result["source_documents"]:
                            context_list.append(doc.page_content)  # 提取文档内容

                        # 显示来源
                        print("来源文档:")
                        sources = set()
                        for doc in answer_result["source_documents"]:
                            if "source" in doc.metadata:
                                sources.add(doc.metadata["source"])
                        for source in sources:
                            print(f"- {source}")

                        # 创建要保存的记录
                        qa_record = {
                            "pdf_name": pdf_name,
                            "question": clean_question,
                            "category": category.strip(),
                            "answer": answer_result['answer'],
                            "context": context_list,
                            "sources": list(sources)
                        }

                        # 写入JSONL文件
                        f.write(json.dumps(qa_record, ensure_ascii=False) + '\n')

        print(f"\n问答记录已保存到: {qa_output_file}")
    else:
        print("无法生成回答：RAG智能体未创建或没有找到之前生成的问题")

except Exception as e:
    print(f"生成回答时出错: {str(e)}")
    import traceback
    traceback.print_exc()


回答生成的问题并保存为JSONL格式:

处理来自 14.4_keV迈克耳孙X...线干涉仪设计及核心器件表征_司尚禹.pdf 的问题:

question: 上海光源作为一台中能第三代同步辐射光源，其主要技术优势在哪里？


C:\Users\lamonki\AppData\Local\Temp\ipykernel_38232\1908086068.py:29: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  category = classification_chain.run(question=clean_question)


分类: General QA


KeyboardInterrupt: 

In [48]:
import json

# 输入文件和输出文件路径
input_file = 'pdf_qa_results.jsonl'
output_file = 'result_output.jsonl'

# 用于记录已出现的 question
seen_questions = set()

# 打开输入文件和输出文件
with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    # 逐行读取输入文件
    for line in infile:
        # 解析 JSON 对象
        data = json.loads(line.strip())
        question = data.get('question')
        answer = data.get('answer')

        # 如果 question 未出现过
        if question and question not in seen_questions:
            # 记录该 question
            seen_questions.add(question)
            # 写入新的 JSON 对象到输出文件
            json.dump({'question': question, 'answer': answer}, outfile,ensure_ascii=False)
            outfile.write('\n')  # 每行一个 JSON 对象

print(f"去重完成，结果已保存到 {output_file}")



去重完成，结果已保存到 result_output.jsonl


In [20]:
import json
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 定义提示词模板，要求生成多个新问题
generate_new_questions_prompt_template = """
请根据原始答案和问题生成两个与原问题同义的新问题，确保语义不变。每个新问题单独占一行:

原问题: {original_question}
原答案: {answer}
新问题:
"""

# 初始化提示词模板
generate_new_questions_prompt = PromptTemplate(
    template=generate_new_questions_prompt_template,
    input_variables=["original_question", "answer"]
)

# 初始化LLMChain
generate_new_questions_chain = LLMChain(
    llm=llm,
    prompt=generate_new_questions_prompt
)

# 定义处理函数
def process_question_answer_pair(data, seen_questions, outfile):
    """处理单个问答对，生成多个新问题并写入文件"""
    original_question = data.get('question')
    answer = data.get('answer')

    # 检查输入数据是否有效
    if not original_question or not answer:
        print(f"无效的输入数据: {data}")
        return

    # 如果原问题未出现过，写入文件
    if original_question not in seen_questions:
        seen_questions.add(original_question)
        json.dump({'question': original_question, 'answer': answer}, outfile, ensure_ascii=False)
        outfile.write('\n')
        print(f"原始问题：{original_question}\n")

    # 生成多个新问题
    try:
        new_questions_output = generate_new_questions_chain.run({
            "original_question": original_question,
            "answer": answer
        }).strip()

        # 解析生成的新问题（假设每个问题单独占一行）
        new_questions = [q.strip() for q in new_questions_output.split('\n') if q.strip()]

        # 处理每个新问题
        for new_question in new_questions:
            if new_question and new_question not in seen_questions:
                seen_questions.add(new_question)
                json.dump({'question': new_question, 'answer': answer}, outfile, ensure_ascii=False)
                outfile.write('\n')
                print(f"新问题:{new_question}\n")
    except Exception as e:
        print(f"生成新问题时出错: {e}")

# 主函数
def process_file(input_file, output_file):
    """处理输入文件并生成输出文件"""
    seen_questions = set()  # 用于记录已处理过的问题

    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        for line in infile:
            try:
                data = json.loads(line.strip())  # 解析JSON数据
                process_question_answer_pair(data, seen_questions, outfile)
            except json.JSONDecodeError as e:
                print(f"JSON解析错误: {e}")
            except Exception as e:
                print(f"处理行时出错: {e}")

# 运行程序
input_file = 'result_output.jsonl'
output_file = 'question_new_output.jsonl'
process_file(input_file, output_file)


原始问题：上海光源作为一台中能第三代同步辐射光源，其主要技术优势在哪里？

新问题:上海光源作为一台中能第三代同步辐射光源，具有哪些关键的技术优势？

新问题:上海光源在中能第三代同步辐射光源领域，其主要的技术优势是什么？

原始问题：同步辐射光源与自由电子激光相比，有什么不同之处？

新问题:同步辐射与自由电子激光在科学研究中的不同应用场景是什么？

新问题:同步辐射光源与自由电子激光在产生原理、谱宽特性及时间分辨率方面有哪些关键区别？

原始问题：上海光源的光束线站如何为各个学科的研究提供支持？

新问题:上海光源如何为不同领域的科学研究提供必要的技术支持？

新问题:上海光源的光束线站在支持多学科研究方面有什么作用？

原始问题：什么是上海光源的波长范围，以及它如何应用于材料科学等领域？

新问题:上海光源的能量范围是什么，它在材料科学、生物医学等领域中的应用前景如何？

新问题:上海同步辐射光源的波长范围是多少，如何利用它来开展材料结构分析和生物医学研究？

原始问题：上海光源在生命科学、凝聚态物理和化学等方面有哪些典型的应用案例？

新问题:上海光源在生物医学研究和材料分析方面有什么样的成功案例？

新问题:上海光源的同步辐射技术在结构生物学和材料科学领域有哪些具体应用？

原始问题：上海光源的加速器系统是如何设计和维护的，以保证其高稳定性和准确性？

新问题:上海光源如何通过加速器系统设计和维护来实现高稳定性和准确性？

新问题:上海光源为了保证其加速器系统具有高稳定性和准确性，采取了哪些具体的设计和调试措施？

原始问题：上海光源在推动我国大科学装置发展方面扮演了怎样的角色？

新问题:上海光源在中国大型科学设施发展中发挥了什么作用？

新问题:上海光源如何为中国的大科学装置建设做出贡献？

原始问题：上海光源的同步辐射光束线站主要用于哪些领域的研究？

新问题:上海光源的同步辐射光束线站主要应用于哪些科学领域的研究？

新问题:上海光源的同步辐射光束线站在材料科学和生命科学等领域中有什么作用？

原始问题：什么是上海光源中加速器的作用，以及它如何产生高能电子束？

新问题:上海光源中使用了什么样的加速系统来生产高能电子束，并且它是如何工作的？

新问题:上海光源中的加速器是如何利用直线加速器、增强器和储存环来产生高能量同步辐射光的？

原始问题：上海

## 补全推理过程

In [14]:
# 构建Agent来补全推理过程
try:
    from langchain.chat_models import ChatOpenAI
    from langchain.schema import HumanMessage, SystemMessage
    from langchain.prompts import ChatPromptTemplate
    import json
    import os

    # 定义输入和输出文件
    qa_input_file = "pdf_qa_results.jsonl"  # 之前保存的问答结果文件
    reasoning_output_file = "qa_with_reasoning.jsonl"  # 包含推理过程的输出文件

    # 检查输入文件是否存在
    if not os.path.exists(qa_input_file):
        print(f"错误：找不到输入文件 {qa_input_file}")
    else:
        # 初始化大模型
        llm = ChatOpenAI(
            temperature=0.1,
            model_name="deepseek-ai/DeepSeek-R1"
            )

        # 创建提示模板
        system_message = """你是一个专业的学术助手，精通各学科领域的专业知识。你的任务是为给定的问题和答案补充详细、准确的推理过程，展示从问题到答案的完整思考路径。

请注意以下要求：
1. 推理过程必须清晰展示从问题分析到得出答案的完整思考链条
2. 基于可靠的学术原理和专业知识，不要编造信息
3. 包含具体的技术细节、公式、原理和机制解释，而非泛泛而谈
4. 使用该领域的专业术语和标准表达方式
5. 按照逻辑顺序组织内容，可以使用编号或小标题增强结构性
6. 必要时引入定量分析，包括数值范围、单位和量级
7. 解释关键参数之间的关系和影响机制
8. 明确说明每一步推理如何引导到下一步，最终如何得出答案中的结论

推理过程应该结构清晰，包含以下部分：
- 问题分析：明确问题的核心要点和需要解决的关键问题
- 技术原理：相关的专业知识和技术原理
- 工作机制：关键组件的工作机制和参数关系
- 系统设计：系统设计中的关键考量和技术路线
- 性能分析：系统性能指标和应用价值

避免使用"XX"这样的占位符，确保所有内容都是具体、明确的。不要使用模板化的结构，而是根据具体问题提供有针对性的分析。"""

        # 打开输入文件读取问答记录
        qa_records = []
        with open(qa_input_file, 'r', encoding='utf-8') as f:
            for line in f:
                qa_records.append(json.loads(line.strip()))

        print(f"读取了 {len(qa_records)} 条问答记录")

        # 如果DEBUG模式，只处理第一条记录
        if DEBUG:
            qa_records = qa_records[:1]
            print("DEBUG模式：只处理第一条记录")

        # 打开输出文件准备写入
        with open(reasoning_output_file, 'w', encoding='utf-8') as f:
            # 遍历每条问答记录
            for i, record in enumerate(qa_records):
                print(f"\n处理第 {i+1}/{len(qa_records)} 条记录...")

                # 获取问题和答案
                question = record["question"]
                answer = record["answer"]

                print(f"question: {question[:50]}..." if len(question) > 50 else f"question: {question}")
                print(f"答案: {answer[:100]}..." if len(answer) > 100 else f"答案: {answer}")

                # 构建提示并调用大模型
                human_message = f"""question: {question}

答案: {answer}

请为上述问答补充详细的推理过程，确保清晰展示从问题到答案的完整思考路径，包含足够的专业深度和技术细节。"""

                messages = [
                    SystemMessage(content=system_message),
                    HumanMessage(content=human_message)
                ]

                # 打印提示信息
                print("发送到LLM的提示:")
                for msg in messages:
                    print(f"[{msg.type}]: {msg.content[:100]}..." if len(msg.content) > 100 else f"[{msg.type}]: {msg.content}")
                print("-" * 50)

                # 获取推理过程
                response = llm.invoke(messages)
                reasoning = response.content

                # 更新记录，添加推理过程
                record["reasoning"] = reasoning

                # 写入JSONL文件
                f.write(json.dumps(record, ensure_ascii=False) + '\n')

                # 打印推理过程
                print(f"推理过程:\n{reasoning}")
                print(f"已添加推理过程 ({len(reasoning)} 字符)")

        print(f"\n推理过程已添加并保存到: {reasoning_output_file}")
        if DEBUG:
            print("注意：由于DEBUG模式开启，只处理了第一条记录")

except Exception as e:
    print(f"补全推理过程时出错: {str(e)}")
    import traceback
    traceback.print_exc()



补全推理过程时出错: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0.1, 'mod...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


C:\Users\lamonki\AppData\Local\Temp\ipykernel_8008\2492814815.py:18: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
Traceback (most recent call last):
  File "C:\Users\lamonki\AppData\Local\Temp\ipykernel_8008\2492814815.py", line 18, in <module>
    llm = ChatOpenAI(
          ^^^^^^^^^^^
  File "C:\Users\lamonki\.conda\envs\Myenvs\Lib\site-packages\langchain_core\_api\deprecation.py", line 214, in warn_if_direct_instance
    return wrapped(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lamonki\.conda\envs\Myenvs\Lib\site-packages\langchain_core\load\serializable.py", line 125, in __init__
    super().__init__(*args, **kwargs)
  File "C